In [2]:
import cv2
from ultralytics import YOLO

def main():
    # Load YOLOv8 model (you can change the weights file if needed)
    model = YOLO("yolov8n.pt")

    # Open webcam (0 = default camera)
    cap = cv2.VideoCapture(0)

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # Run detection with confidence threshold 0.4
        results = model(frame, conf=0.4, verbose=False)

        # Loop through each result
        for r in results:
            boxes = r.boxes  # detected boxes
            ids = boxes.id   # track IDs (can be None)

            # Convert ids to numpy if not None
            if ids is not None:
                ids = ids.cpu().numpy().astype(int)

            # Draw detections on frame
            for i, box in enumerate(boxes):
                # Get bounding box coordinates
                x1, y1, x2, y2 = map(int, box.xyxy[0])

                # Class name
                cls = model.names[int(box.cls[0])]

                # Track ID if available
                obj_id = ids[i] if ids is not None and i < len(ids) else None

                # Prepare label text
                label = f"{cls}"
                if obj_id is not None:
                    label += f" ID:{obj_id}"

                # Draw rectangle and label on frame
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                cv2.putText(frame, label, (x1, y1 - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

        # Show the frame
        cv2.imshow("YOLOv8 Detection", frame)

        # Press 'q' to quit
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break

    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()